In [1]:
#dbutils.fs.mount(
#  source = "wasbs://CONTENEDOR@BLOB.blob.core.windows.net/",
#  mount_point = "/mnt/NOMBRE_MONTADO",
#  extra_configs = {"fs.azure.account.key.BLOB.blob.core.windows.net":"TOKEN"})

In [ ]:
df=spark.read.format('csv').option('header',True).option('inferSchema',True).load('/mnt/containing/vuelo')

In [ ]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import *

vuelos= df.toPandas()

In [ ]:
# CAMBIANDO LOS NOMBRES DE TODAS LAS COLUMNAS

vuelos.rename(columns={'Year':'Anio'}, inplace=True)
vuelos.rename(columns={'Month':'Mes'}, inplace=True)
vuelos.rename(columns={'DayofMonth':'Dia'}, inplace=True)
vuelos.rename(columns={'DayOfWeek':'Dia_Sem'}, inplace=True)
vuelos.rename(columns={'OP_CARRIER':'ID_Aerolinea'}, inplace=True)
vuelos.rename(columns={'ORIGIN':'ID_Aeropuerto_O'}, inplace=True)
vuelos.rename(columns={'DEST':'ID_Aeropuerto_D'}, inplace=True)
vuelos.rename(columns={'DEP_TIME':'Hora_SalidaR'}, inplace=True)
vuelos.rename(columns={'CRS_DEP_TIME':'Hora_SalidaP'}, inplace=True)
vuelos.rename(columns={'ARR_TIME':'Hora_LlegadaR'}, inplace=True)
vuelos.rename(columns={'CRS_ARR_TIME':'Hora_LlegadaP'}, inplace=True)
vuelos.rename(columns={'CRS_ELAPSED_TIME':'Duracion_Promedio'}, inplace=True)
vuelos.rename(columns={'ARR_DELAY':'Retraso_Llegada'}, inplace=True)
vuelos.rename(columns={'DEP_DELAY':'Retraso_Salida'}, inplace=True)
vuelos.rename(columns={'DISTANCE':'Distancia'}, inplace=True)
vuelos.rename(columns={'CANCELLED':'Cancelado'}, inplace=True)
vuelos.rename(columns={'CANCELLATION_CODE':'Codigo_Cancelacion'}, inplace=True)
vuelos.rename(columns={'CARRIER_DELAY':'Retraso_Aerolinea'}, inplace=True)
vuelos.rename(columns={'WEATHER_DELAY':'Retraso_Clima'}, inplace=True)
vuelos.rename(columns={'NAS_DELAY':'Retraso_NAS'}, inplace=True)
vuelos.rename(columns={'SECURITY_DELAY':'Retraso_Seguridad'}, inplace=True)
vuelos.rename(columns={'LATE_AIRCRAFT_DELAY':'Retraso_Aeronave'}, inplace=True)

# Reordenamos las columnas
vuelos = vuelos[['Anio', 'Mes', 'Dia', 'Dia_Sem', 'ID_Aerolinea', 'ID_Aeropuerto_O', 'ID_Aeropuerto_D', 'Hora_SalidaR',
 'Hora_SalidaP', 'Hora_LlegadaR', 'Hora_LlegadaP', 'Duracion_Promedio', 'Retraso_Llegada', 'Retraso_Salida', 'Distancia', 
 'Cancelado', 'Codigo_Cancelacion', 'Retraso_Aerolinea', 'Retraso_Clima', 'Retraso_NAS', 'Retraso_Seguridad', 
 'Retraso_Aeronave']]

In [ ]:
import numpy as np

# Crear una columna 'Estado_Vuelo', que tome valores 'Codigo_Cancelacion'
vuelos['ID_EstadoVuelo'] = np.where(vuelos['Codigo_Cancelacion'].notnull(), vuelos['Codigo_Cancelacion'], np.where(vuelos['Cancelado'] == 1, 'ND', 'E'))

# Eliminar columna 'Codigo_Cancelacion'
vuelos.drop('Codigo_Cancelacion', axis=1, inplace=True)
vuelos.drop('Cancelado', axis=1, inplace=True)

# Eliminar las columnas de retrasos
vuelos.drop('Retraso_Aerolinea', axis=1, inplace=True)
vuelos.drop('Retraso_NAS', axis=1, inplace=True)
vuelos.drop('Retraso_Seguridad', axis=1, inplace=True)
vuelos.drop('Retraso_Aeronave', axis=1, inplace=True) 
vuelos.drop('Retraso_Clima', axis=1, inplace=True)


# Creando una columna de 'Partida_A_Tiempo', A: 'Retraso_Salida' <= 0, B: 0 < 'Retraso_Salida' <= 15, C: 'Retraso_Salida' > 15, NE: 'Retraso_Salida' = NaN
vuelos['ID_DepTime'] = np.where(vuelos['Retraso_Salida'] <= 0, 'A', np.where(vuelos['Retraso_Salida'] <= 15, 'B', np.where(vuelos['Retraso_Salida'] > 15, 'C', 'ND')))

# Creando una columna de 'Llegada_A_Tiempo', A: 'Retraso_Llegada' <= 0, B: 0 < 'Retraso_Llegada' <= 15, C: 'Retraso_Llegada' > 15, NE: 'Retraso_Llegada' = NaN
vuelos['ID_ArrTime'] = np.where(vuelos['Retraso_Llegada'] <= 0, 'A', np.where(vuelos['Retraso_Llegada'] <= 15, 'B', np.where(vuelos['Retraso_Llegada'] > 15, 'C', 'ND')))

# Eliminar las columnas de retrasos
vuelos.drop('Retraso_Llegada', axis=1, inplace=True)
vuelos.drop('Retraso_Salida', axis=1, inplace=True)

vuelos = vuelos.sample(frac=0.02, random_state=1)

In [ ]:
S_vuelo = spark.createDataFrame(vuelos)

# if not spark.catalog.tableExists("vuelo2018"):
#     S_vuelo.write.saveAsTable("vuelo2018")



In [ ]:
%sql
SELECT * FROM vuelo2018

In [ ]:
jdbcHostname = "SERVIDOR.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "BASE DE DATOS"
properties = {
    "user" : "USUARIO",
    "password" : "CONTRASEÑA",
}
url = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

In [ ]:
# Conectar a SQL Server la tabla de vuelos
if not spark.catalog.tableExists("v2018"):
    v2018 = spark.sql("SELECT * FROM vuelo2018")
    V2018 = DataFrameWriter(v2018)
    V2018.jdbc(url=url, table="dbo.V2018", mode="overwrite", properties=properties)